In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir('C:\\Users\\Usuario\\TFG\\digipanca\\')

In [4]:
import torch
from src.models.custom_deeplabv3 import CustomDeepLabV3 as dl
from src.metrics.segmentation import SegmentationMetrics as sm
from src.metrics.segmentation_bak import SegmentationMetrics as smb

# __Test__

In [11]:
B, num_classes, H, W = 4, 5, 256, 256
model = dl(num_classes=num_classes, dropout_rate=0.2)

## Generate prediction (_logits_)

In [32]:
channels = 1
input_tensor = torch.randn(B, channels, H, W)
output = model(input_tensor)['out']
print(output.shape)

torch.Size([4, 5, 256, 256])


## Generate random mask (_class indices_)

In [33]:
random_mask = torch.randint(0, num_classes, (B, H, W))
print(random_mask.shape)

torch.Size([4, 256, 256])


### Test

In [34]:
metrics_a = sm.all_metrics(output, random_mask)
metrics_b = smb.all_metrics(output, random_mask)

else if


In [35]:
print("EQUALS:", (metrics_a==metrics_b))
print('-'*40)
print(metrics_a)
print('-'*40)
print(metrics_b)

EQUALS: False
----------------------------------------
{'dice_class_0': 0.0778409019112587, 'dice_class_1': 0.1709488034248352, 'dice_class_2': 0.2024432122707367, 'dice_class_3': 0.13893862068653107, 'dice_class_4': 0.1975739300251007, 'dice_mean': 0.15754908323287964, 'iou_class_0': 0.041024453938007355, 'iou_class_1': 0.09694793075323105, 'iou_class_2': 0.11460772156715393, 'iou_class_3': 0.07773466408252716, 'iou_class_4': 0.11103041470050812, 'iou_mean': 0.08826903998851776, 'precision_class_0': 0.19764210283756256, 'precision_class_1': 0.2007494568824768, 'precision_class_2': 0.20040568709373474, 'precision_class_3': 0.1935102343559265, 'precision_class_4': 0.2019074708223343, 'recall_class_0': 0.05310937389731407, 'recall_class_1': 0.2837788164615631, 'recall_class_2': 0.2777024209499359, 'recall_class_3': 0.16389887034893036, 'recall_class_4': 0.2251136153936386, 'dice': 0.15754908323287964, 'iou': 0.08826903998851776, 'precision': 0.19884298741817474, 'recall': 0.2007206231355

In [31]:
def to_one_hot(masks, num_classes):
    if masks.dim() == 3:  # 2D (batch_size, height, width)
        batch_size, height, width = masks.shape
        masks_one_hot = torch.zeros(batch_size, num_classes, height, width, device=masks.device)
        masks_one_hot.scatter_(1, masks.unsqueeze(1), 1)

    return masks_one_hot

import monai.transforms as MT
def monai_to_one_hot(y, N):
    ad = MT.AsDiscrete(argmax=True, to_onehot=N)
    y_oh = ad(y)
    return y_oh

In [41]:
import monai.metrics as mm

# y_pred = torch.argmax(output, dim=1, keepdim=True)
# y_pred = torch.nn.functional.softmax(output, dim=1)
y_pred = torch.argmax(output, dim=1, keepdim=True)
y_pred_one_hot = torch.zeros(B, num_classes, H, W)
y_pred_one_hot.scatter_(1, y_pred, 1)

y_true = to_one_hot(random_mask, num_classes)
dice_monai = mm.DiceMetric(reduction='none')(y_pred_one_hot, y_true)

print(dice_monai.mean(dim=0))
print(dice_monai.mean(dim=0).mean())

tensor([0.0778, 0.1709, 0.2024, 0.1389, 0.1976])
tensor(0.1575)


In [42]:
perfect_pred = y_true.clone()
pd_a = sm.dice_coefficient(perfect_pred, y_true)
pd_b = smb.dice_coefficient(perfect_pred, y_true)

print("EQUALS:", (pd_a==pd_b))
print('-'*40)
print(pd_a)
print('-'*40)
print(pd_b)

else
EQUALS: True
----------------------------------------
(tensor(1.), {'dice_class_0': 1.0, 'dice_class_1': 1.0, 'dice_class_2': 1.0, 'dice_class_3': 1.0, 'dice_class_4': 1.0, 'dice_mean': 1.0})
----------------------------------------
(tensor(1.), {'dice_class_0': 1.0, 'dice_class_1': 1.0, 'dice_class_2': 1.0, 'dice_class_3': 1.0, 'dice_class_4': 1.0, 'dice_mean': 1.0})
